Copyright © 2021 HQS Quantum Simulations GmbH. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except
in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.

# Deutsch-Josza Algorithm
We have given a function `f: {0, 1}^n->{0, 1}` from input bitstrings with length `n`, e.g., `00110`, to a single bit output. The given function is either balanced (50% of inputs yield 0, 50% yield 1 as output) or the function is constant (alwasy 1 or always 0). The task for our algorithm is to decide if the given function is constant or balanced. 

On a conventional computer we can query the function using the different input bitstrings. As soon as we have seen two different outputs we know that the function is balanced. However, if we have measured k-times the same value we only know that the function is constant with probability `P_k=1-1/2^(k-1)`. If we want to be 100% certain we need to query 50% of all `2^n` bitstrings, i.e., `2^(n-1)` queries.

The Deutsch-Josza algorithm can perform the same task with exactly 1 query using `n+1` qubits.

![Deutsch Josza](deutsch_josza.png)

The large block in the center of the algorithm is an oracle.

## Oracles

The oracle `U_f` takes two input registers `|x>` and `|y>` and returns $|x\rangle|y\oplus f(x)\rangle$ where $y\oplus f(x)$ is understood as addition modulo 2. For example, $0\oplus1=1$ and $1\oplus1=0$.

In our example we will us a function defined on 2 (q)bits `f: {0,1}^2->{0,1}`. We can define a balanced oracle as

![balanced oracle](oracle_balanced.png)

and a constant oracle as

![constant oracle](oracle_constant_1.png)

## Implementation

### Initialization
Required is the first step of the circuit: Flip the 3rd qubit from `0 ` to `1` and apply a Hadamard gate to all 3 qubits. We use the qoqo toolkit to represent quantum circuits.

A `Circuit` is the main class to represent quantum circuits. The `qoqo.operations` module contains one- and two-qubit operations such as Hadamard, PauliX or CNOT. For the initialization circuit we require two different gates, the `PauliX` and the `Hadamard` operation.

The following code imports required tools from qoqo and writes a circuit that applies the required operations.

In [2]:
from typing import List

def check_constant(res: List[bool]) -> bool:
    return all([not _ for _ in res])

In [3]:
from qoqo import Circuit
from qoqo import operations as ops

def deutsch_josza(number_qubits: int, oracle: Circuit) -> Circuit:
    circuit = Circuit()
    circuit += ops.PauliX(number_qubits)
    for q in range(number_qubits):
        circuit += ops.Hadamard(q)
    circuit += ops.Hadamard(number_qubits)
    circuit += oracle
    for q in range(number_qubits):
        circuit += ops.Hadamard(q)
    return circuit


### Implement oracles
For the second step of the circuit we require the oracles. The code implements a circuit for the balanced oracle

![Balanced](oracle_balanced.png)

and a circuit for the constant oracle

![Constant](oracle_constant_1.png)

For the balanced oracle a `CNOT` operation which takes a `control` and a `target` qubit is required.

In [4]:
def balanced_oracle(number_qubits: int) -> Circuit:
    oracle = Circuit()
    for c in range(number_qubits):
        oracle += ops.CNOT(control=c, target=number_qubits)
    return oracle

def constant_1_oracle(number_qubits: int) -> Circuit:
    oracle = Circuit()
    oracle += ops.PauliX(number_qubits)
    return oracle

### Finalize the circuit
We define a measurement circuit that 
1. applies a Hadamard gate to the first two qubits
2. defines a bit register
3. applies a `MeasureQubit` operation to the first two qubits

For this step we require two additional qoqo operations: 

* `DefinitionBit` - Create a classical bit register to store measured bit values.
* `MeasureQubit` -  Measure a qubit and store the input in the classical bit register.

In [5]:
number_qubits = 2
balanced = Circuit()
balanced += deutsch_josza(
    number_qubits, oracle=balanced_oracle(number_qubits)
)
balanced += ops.DefinitionBit('ro', number_qubits, is_output=True)
for q in range(number_qubits):
    balanced += ops.MeasureQubit(q, 'ro', q)
print(balanced)

constant = Circuit()
constant += deutsch_josza(
    number_qubits, oracle=constant_1_oracle(number_qubits)
)
constant += ops.DefinitionBit('ro', number_qubits, is_output=True)
for q in range(number_qubits):
    constant += ops.MeasureQubit(q, 'ro', q)
print(constant)

DefinitionBit(DefinitionBit { name: "ro", length: 2, is_output: true })
PauliX(PauliX { qubit: 2 })
Hadamard(Hadamard { qubit: 0 })
Hadamard(Hadamard { qubit: 1 })
Hadamard(Hadamard { qubit: 2 })
CNOT(CNOT { control: 0, target: 2 })
CNOT(CNOT { control: 1, target: 2 })
Hadamard(Hadamard { qubit: 0 })
Hadamard(Hadamard { qubit: 1 })
MeasureQubit(MeasureQubit { qubit: 0, readout: "ro", readout_index: 0 })
MeasureQubit(MeasureQubit { qubit: 1, readout: "ro", readout_index: 1 })

DefinitionBit(DefinitionBit { name: "ro", length: 2, is_output: true })
PauliX(PauliX { qubit: 2 })
Hadamard(Hadamard { qubit: 0 })
Hadamard(Hadamard { qubit: 1 })
Hadamard(Hadamard { qubit: 2 })
PauliX(PauliX { qubit: 2 })
Hadamard(Hadamard { qubit: 0 })
Hadamard(Hadamard { qubit: 1 })
MeasureQubit(MeasureQubit { qubit: 0, readout: "ro", readout_index: 0 })
MeasureQubit(MeasureQubit { qubit: 1, readout: "ro", readout_index: 1 })



## Simulation

The algorithm is then tested on a (simulated) quantum computer. We use the `qoqo_quest` library to run the simulation. From the library you need the `Backend`.
A circuit can be simulated on the backend using `Backend(n_qubits).run_circuit`. The method returns a tuple. The first entry of the tuple is a dictionary of BitRegisters. The result of `run_circuit` is saved into `res`, we then access our registry via `res[0]['ro']`.

The follwing code runs the simulation for the balanced and the constant oracle.
  

In [6]:
import qoqo_quest

print("running balanced")
(res, _, _) = qoqo_quest.Backend(number_qubits+1).run_circuit(balanced)
print(res)
print("is constant?", check_constant(res['ro'][0]))

print("running constant")
(res, _, _) = qoqo_quest.Backend(number_qubits+1).run_circuit(constant)
print(res)
print("is constant?", check_constant(res['ro'][0]))

running balanced
{'ro': [[True, True]]}
is constant? False
running constant
{'ro': [[False, False]]}
is constant? True


### Interpreting the result
The Deutsch-Josza algorithm uses destructive interference to suppress all amplitues but the $|00\rangle$ state if the function is constant. This means that in the constant case you will alway measure `[False, False]` as a result of the circuit. The following diagram shows how the prefactors of different base states change with each gate during the algorithm. Orange arrows hint at a sign change. Notice that the right bit represents the ancilla qubit that is not measured.

![Constant Interference](constant.png)

For the balanced oracle the interference suppresses the amplitue of the $|00\rangle$ state. This means that you can measure all bitstrings but `[False, False]`. 
![Balanced Interference](balanced.png)

This means that measuring `[False, False]` means that the function is constant while all other results mean that the function is balanced.